## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("cameronsobell@gmail.com")  # Change to your verified sender
    to_email = To("cameronsobell@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [31]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. 

Navigating the complexities of SOC 2 compliance can be a daunting task, especially with the ever-evolving requirements. At ComplAI, we specialize in streamlining this process, helping businesses like yours achieve and maintain compliance with minimal effort.

Our AI-powered SaaS tool offers:

- **Automated Documentation**: Reduce time spent on paperwork with smart templates tailored to your needs.
- **Real-Time Monitoring**: Stay ahead of compliance requirements with alerts and updates relevant to your industry.
- **Seamless Audit Preparation**: Simplify audits with organized data and easy access to necessary documentation.

We understand that each organization has unique challenges, and I’d love to discuss how ComplAI can specifically assist [Company Name] in ensuring a smooth compliance journey.

Would you be open to a brief call this week to explore how we can 

In [32]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance with AI-Powered Solutions

Hi [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance and audit preparations can be daunting. At ComplAI, we understand the challenges businesses face in maintaining compliance while ensuring operational efficiency. Our AI-powered SaaS tool simplifies this process, enabling teams like yours to effortlessly manage documentation, track requirements, and streamline audits.

With our solution, you can:

- Automate compliance tasks, reducing manual effort and minimizing human error
- Access real-time reporting and analytics to stay ahead of compliance requirements
- Ensure continuous monitoring of your controls, adapting to changes in the regulatory landscape

We have helped numerous organizations like yours save both time and resources, ultimately leading to smoother audits and enhanced compliance posture.

I would love to schedule a brief call to discuss how ComplAI ca

In [6]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [34]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Let’s Make Compliance as Fun as a Day at the Beach! 🏖️

Hi [Recipient's Name],

I hope this email finds you enjoying your coffee (or your sixth cup, no judgment here!). I wanted to pop into your inbox with something that might be more exciting than your daily to-do list — SOC 2 compliance!

Now, I know what you’re thinking: “Compliance? Fun? Surely this is a typo.” But fear not! At ComplAI, we’ve taken the stress out of compliance and turned it into a smooth ride powered by AI. Imagine having a trusty sidekick that handles audits, keeps your data secure, and lets you focus on what you do best — like planning that beach vacation you deserve!

**Here’s why we’re the sunscreen to your compliance sunburn:**

- **Automated Review:** No more manual drudgery; let our AI do the heavy lifting.
- **Instant Insights:** Know your compliance status in real-time without breaking a sweat.
- **Stress-Free Audits:** Walk into audits like you own the place (because you will!).

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [7]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [8]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [9]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("cameronsobell@gmail.com")  # Change to your verified sender
    to_email = To("cameronsobell@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [10]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x103dad9e0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [11]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b24f600>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [12]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b24e480>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b24e340>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

## And now it's time for our Sales Manager - our planning agent

In [13]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [14]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [15]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("cameronsobell@gmail.com")  # Change to your verified sender
    to_email = To("cameronsobell@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [16]:
tools = [subject_tool, html_tool, send_html_email]

In [17]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10a0b0cc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10a091080>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales pros

In [18]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [19]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b24e480>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b24e340>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [21]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Part 3: Reply Handling

> Goal: capture replies, classify intent, and have an agent draft/return responses. We use SendGrid Inbound Parse (or any POST webhook) to hit a FastAPI endpoint we run inside the notebook, then hand the payload to new agents.

### Unified GMAIL API Approach

#### Gmail


In [22]:
!uv pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

Using Python 3.12.9 environment at: /Users/cameronbell/Projects/agents/.venv
Audited 4 packages in 34ms


#### Authenticate GMAIL API

In [24]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import os

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    """
    Authenticate with Gmail API using OAuth 2.0.
    Run once to generate token.json.
    """
    creds = None
    
    # Load existing token if available
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If no valid credentials, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                print("🔄 Token refreshed successfully")
            except Exception as e:
                print(f"⚠️  Token refresh failed: {e}")
                print("🔐 Starting new authentication flow...")
                creds = None
        
        if not creds:
            # Start OAuth flow (requires credentials.json from Google Cloud Console)
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', 
                SCOPES,
                redirect_uri='http://localhost:0'  # More explicit for 2025
            )
            creds = flow.run_local_server(
                port=0,
                authorization_prompt_message='Opening browser for Gmail authorization...',
                success_message='Authentication successful! You can close this window.',
                open_browser=True
            )
        
        # Save credentials for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        print("✅ Authentication successful! token.json saved.")
    else:
        print("✅ Using existing valid credentials from token.json")
    
    return creds

# Run this once
authenticate_gmail()

Opening browser for Gmail authorization...
✅ Authentication successful! token.json saved.


In [30]:
import base64
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import time
from typing import Dict, Any

# Define the required scopes (make sure this is consistent everywhere)
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

# Helper function to find the plain text body from the complex MIME payload
def get_email_body(payload: Dict[str, Any]) -> str:
    """Recursively search payload parts for the plain text email body."""
    parts = payload.get('parts')
    if parts:
        for part in parts:
            if part.get('mimeType') == 'text/plain':
                data = part['body'].get('data')
                if data:
                    return base64.urlsafe_b64decode(data).decode('utf-8')
            # Check nested parts
            result = get_email_body(part)
            if result:
                return result
    
    # Fallback for simple message structure
    data = payload.get('body', {}).get('data')
    if data:
        return base64.urlsafe_b64decode(data).decode('utf-8')
        
    return ""

@function_tool
def check_gmail_for_replies_by_thread(thread_id: str, wait_seconds: int = 30):
    """
    Check Gmail inbox for UNREAD replies within a specific conversation thread.
    Waits up to wait_seconds for a reply.
    """
    if not thread_id:
        return {"reply_found": False, "message": "Error: Missing thread_id."}

    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    service = build('gmail', 'v1', credentials=creds)
    
    end_time = time.time() + wait_seconds
    while time.time() < end_time:
        # Search for UNREAD messages within the specified thread
        # The query 'is:unread' ensures we only pick up the prospect's reply
        results = service.users().messages().list(
            userId='me',
            q=f'threadId:{thread_id} is:unread'
        ).execute()
        
        messages = results.get('messages', [])
        if messages:
            # Get the ID of the first *new* message in the thread
            new_message_id = messages[0]['id']
            
            # Fetch the new message data
            msg = service.users().messages().get(
                userId='me', 
                id=new_message_id
            ).execute()
            
            # Extract email body using the helper function
            decoded_body = get_email_body(msg['payload'])
            
            # Extract sender's email (a header search is often required for 'From')
            from_header = next((h['value'] for h in msg['payload']['headers'] if h['name'] == 'From'), 'Unknown Sender')
            
            # Mark the message as read
            service.users().messages().modify(
                userId='me',
                id=new_message_id,
                body={'removeLabelIds': ['UNREAD']}
            ).execute()
            
            return {
                "reply_found": True,
                "body": decoded_body,
                "thread_id": thread_id,
                "from_email": from_header,
                "subject": msg['snippet'] # Use snippet as a placeholder for subject if needed
            }
        
        time.sleep(5)
    
    return {"reply_found": False, "message": "No reply received"}

#### Gmail Sending Tool

In [27]:
from pydantic import BaseModel
from typing import Optional

In [39]:
from email.mime.text import MIMEText
from googleapiclient.errors import HttpError
import base64
from typing import Dict, Any, Optional

@function_tool
def send_gmail_api_email(to_email: str, subject: str, html_body: str, thread_id: Optional[str] = None) -> Dict[str, Any]:
    """
    Sends an email using the Gmail API, optionally continuing a thread.
    Returns the thread_id of the sent message.
    """
    try:
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        service = build('gmail', 'v1', credentials=creds)
        
        # TESTING: Override recipient for all test emails
        to_email = "cameronsobell@gmail.com"

        # 1. Create the MIME message
        message = MIMEText(html_body, 'html')
        message['to'] = to_email
        message['subject'] = subject
        
        # 2. Add 'References' or 'In-Reply-To' for threading continuity (Optional but good)
        if thread_id:
             # Find the last message in the thread to get the Message-ID for In-Reply-To
             # This is complex, but Gmail API handles basic threading if threadId is set in the body
             pass 

        # 3. Encode and send the message
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        
        body = {'raw': raw_message}
        if thread_id:
             body['threadId'] = thread_id # CRITICAL: Tells Gmail to use the existing thread

        sent_message = service.users().messages().send(
            userId='me', 
            body=body
        ).execute()

        # 4. Return the threadId
        return {
            "status": "success",
            "message_id": sent_message['id'],
            "thread_id": sent_message['threadId'] # <-- Guarantees correct ID
        }

    except HttpError as error:
        return {"status": "error", "message": f"Gmail API error: {error}"}
    except Exception as e:
        return {"status": "error", "message": f"An unexpected error occurred: {e}"}

#### gmail emailer agent


In [40]:
# ============================================
# GMAIL API TOOLS (Not SendGrid)
# ============================================

# 1. Subject writer tool (reuse existing or create Gmail-specific)
gmail_subject_tool = subject_writer.as_tool(
    tool_name="gmail_subject_writer",
    tool_description="Generate subject line for Gmail API email"
)

# 2. HTML converter tool (reuse existing or create Gmail-specific)
gmail_html_tool = html_converter.as_tool(
    tool_name="gmail_html_converter",
    tool_description="Convert email body to HTML for Gmail API"
)

# 3. Gmail sending tool (your new function)
# send_gmail_api_email is already decorated with @function_tool, so just reference it
gmail_send_tool = send_gmail_api_email  # Already a FunctionTool

# 4. Tools list for Gmail Emailer
gmail_tools = [gmail_subject_tool, gmail_html_tool, gmail_send_tool]

In [41]:
# ============================================
# GMAIL EMAIL MANAGER AGENT
# ============================================

gmail_manager_instructions = """
You are the Gmail Email Manager for ComplAI.
You receive a plain text email body and recipient, then format and send via Gmail API.

Steps:
1. Use gmail_subject_writer to generate an engaging subject line
2. Use gmail_html_converter to convert the body to HTML
3. Use send_gmail_api_email to send the email
4. CRITICAL: Return the thread_id from the sent email for conversation tracking

Always complete all three steps.
"""

gmail_email_manager = Agent(
    name="Gmail Email Manager",
    instructions=gmail_manager_instructions,
    tools=gmail_tools,
    model="gpt-4o-mini",
    handoff_description="Format and send emails via Gmail API"
)

In [42]:
# ============================================
# SALES MANAGER (Updated for Gmail)
# ============================================

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts.
2. Evaluate and Select: Review the drafts and choose the single best email.
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Gmail Email Manager' agent.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Gmail Email Manager — never more than one.
- Wait for the Gmail Email Manager to return the thread_id before finishing.
"""

sales_manager_gmail = Agent(
    name="Sales Manager (Gmail)",
    instructions=sales_manager_instructions,
    tools=[tool1, tool2, tool3],  # Your 3 sales agent tools
    handoffs=[gmail_email_manager],  # ← Changed from SendGrid emailer_agent
    model="gpt-4o-mini"
)

In [ ]:
# Test sending via Gmail API
message = "Send a cold sales email to 'Dear CEO from Alice' using Gmail"

with trace("Automated SDR - Gmail"):
    result = await Runner.run(sales_manager_gmail, message)
    print("✅ Email sent via Gmail API")
    print(f"Thread ID: {result.final_output}")  # Should contain thread_id

MaxTurnsExceeded: Max turns (10) exceeded

In [ ]:
# Reuse send_email/send_html_email already defined earlier
reply_router = Agent(
    name="Reply Router",
    instructions="""
You triage inbound replies. Output JSON with keys:
- classification: one of ["positive", "neutral", "negative", "unsubscribe", "question"]
- summary: <=280 character recap
- next_step: either "auto_reply" or "human_review"
""",
    model="gpt-4o-mini"
)

closer_agent = Agent(
    name="Closer Agent",
    instructions="""
You write courteous follow-up replies as Alice from ComplAI.
Only respond if the router says next_step == auto_reply.

Steps:
1. Draft a plain text response body (no subject needed)
2. Hand off the email body to the Email Manager agent to format and send
3. Confirm the handoff was completed

Always hand off to Email Manager - do not try to send directly.
""",
    handoffs=[emailer_agent],  # Changed from tools to handoffs
    model="gpt-4o-mini"
)

conversation_log: list[dict] = []  # simple in-memory store; swap for DB in production

In [ ]:
# UNIFIED WORKFLOW: Automated SDR with Prospect Simulation & Reply Handling

# First, create a conversation orchestrator that manages the full loop
conversation_orchestrator_instructions = """
You are a Conversation Orchestrator for ComplAI's automated SDR system.

Your workflow:
1. Use the sales_manager tool to generate and send the initial cold email
2. Use the prospect_simulator tool to generate a realistic reply
3. Use the reply_handler tool to classify and respond to the prospect's reply

Execute all three steps in sequence. Report a summary at the end.
"""

# Convert existing agents to tools
prospect_tool = prospect_agent.as_tool(
    tool_name="prospect_simulator",
    tool_description="Simulate a prospect's reply to a cold email with specified tone (positive/neutral/negative/question)"
)

# Create a reply handler agent that uses router + closer
reply_handler_instructions = """
You handle prospect replies in two steps:
1. First, classify the reply using reply_router to determine intent and next_step
2. If next_step is "auto_reply", draft and SEND the response using closer_agent
3. Return a summary of what happened

Always invoke the tools to do the work.
"""

reply_handler = Agent(
    name="Reply Handler",
    instructions=reply_handler_instructions,
    tools=[
        reply_router.as_tool("reply_router", "Classify prospect reply intent"),
        closer_agent.as_tool("closer_agent", "Draft and send automated follow-up email")
    ],
    model="gpt-4o-mini"
)

reply_handler_tool = reply_handler.as_tool(
    tool_name="reply_handler",
    tool_description="Triage a prospect reply and send automated response if appropriate"
)

# Create the orchestrator
conversation_orchestrator = Agent(
    name="Conversation Orchestrator",
    instructions=conversation_orchestrator_instructions,
    tools=[
        sales_manager.as_tool("sales_manager", "Generate and send initial cold email"),
        prospect_tool,
        reply_handler_tool
    ],
    model="gpt-4o-mini"
)

# Run the complete workflow in ONE trace
message = """
Execute the full SDR workflow:
1. Have sales_manager send a cold email to 'Dear CEO'
2. Have prospect_simulator generate a 'positive' tone reply to that email
3. Have reply_handler process the reply and send an automated response
"""

with trace("Complete SDR with Reply Automation"):
    result = await Runner.run(conversation_orchestrator, message)
    print("\n=== WORKFLOW COMPLETE ===")
    print(result.final_output)

In [ ]:
# Agent that checks Gmail and handles real replies
real_reply_checker = Agent(
    name="Real Reply Checker",
    instructions="""
    You check Gmail for actual replies to sent emails.
    Use the check_gmail_for_replies tool to poll the inbox.
    If a reply is found, pass it to the reply_handler.
    If no reply is found, report that to the user.
    """,
    tools=[check_gmail_for_replies, reply_handler_tool],
    model="gpt-4o-mini"
)

Crucial Change: The agent must now receive the thread_id to pass to the new tool.

In [ ]:
# Assuming you have defined the reply_router agent
# and the new check_gmail_for_replies_by_thread tool

real_reply_checker = Agent(
    name="Real Reply Checker",
    instructions="""
    You check Gmail for actual replies to sent emails.
    Use the 'check_gmail_for_replies_by_thread' tool with the provided thread_id 
    to poll the inbox.
    
    If a reply is found:
    1. Log the full reply data to the conversation_log.
    2. Pass the full reply data (including reply_found: True, body, thread_id, etc.) 
       to the **Reply Router** handoff for triage and classification.
       
    If no reply is found, report that to the user.
    """,
    tools=[check_gmail_for_replies_by_thread], # Update tool name
    handoffs=[reply_router], # Pass the Reply Router as a handoff
    model="gpt-4o-mini"
)

# How the workflow runs now (assuming cold_email_result contains the thread_id)

# 1. Capture Thread ID (from your send function, not fully shown, but crucial)
# sent_thread_id = cold_email_result['thread_id'] 

# 2. Wait for and handle real replies
print("\n=== STEP 2: Waiting for Real Reply (30 seconds) ===")

with trace("Real Email Reply Flow"):
    reply_result = await Runner.run(
        real_reply_checker,
        f"Check Gmail for a reply in thread {sent_thread_id}" # Agent gets the thread ID
    )
    # ...

In [ ]:
# Assuming you define the target email somewhere
TARGET_PROSPECT_EMAIL = "target@example.com" # Define this for testing

# ... (rest of the setup)

# === STEP 1: Sending Cold Email ===
print("=== STEP 1: Sending Cold Email ===")
with trace("Send Cold Email"):
    # The sales_manager must be prompted to include the recipient
    cold_email_result = await Runner.run(
        sales_manager,
        f"Send out a cold sales email addressed to Dear CEO to {TARGET_PROSPECT_EMAIL} from Alice"
    )
    
# CRITICAL: Extract the thread ID from the result of the initial send
# Note: You need to ensure the Email Manager agent returns this value cleanly.
# Assuming the Email Manager returns the tool's output directly:
sent_thread_id = cold_email_result.final_output['thread_id'] 
print(f"✅ Cold email sent! Tracking Thread ID: {sent_thread_id}")


# === STEP 2: Wait for and handle real replies ===
print("\n=== STEP 2: Waiting for Real Reply (30 seconds) ===")

with trace("Real Email Reply Flow"):
    # Pass the thread_id to the checker agent
    reply_result = await Runner.run(
        real_reply_checker,
        f"Check Gmail for a reply in thread {sent_thread_id}. The initial subject was 'SOC 2 Compliance'."
    )
    print("\n=== WORKFLOW COMPLETE ===")
    print(reply_result.final_output)

In [ ]:
# Step 1: Send the initial cold email
print("=== STEP 1: Sending Cold Email ===")
with trace("Send Cold Email"):
    cold_email_result = await Runner.run(
        sales_manager,
        "Send out a cold sales email addressed to Dear CEO from Alice"
    )
    print("✅ Cold email sent!")

# Step 2: Wait for and handle real replies
print("\n=== STEP 2: Waiting for Real Reply (30 seconds) ===")
print("📧 Manually reply to the email in your Gmail within 30 seconds...")

with trace("Real Email Reply Flow"):
    reply_result = await Runner.run(
        real_reply_checker,
        "Check Gmail for replies to 'SOC 2 Compliance' and handle any responses"
    )
    print("\n=== WORKFLOW COMPLETE ===")
    print(reply_result.final_output)

### Set Up for WebHook (archived)

#### 1.2 Install + extra imports (new code cell immediately after the markdown)

In [49]:
# Only run once
!uv pip install fastapi uvicorn pyngrok nest_asyncio

Using Python 3.12.9 environment at: /Users/cameronbell/Projects/agents/.venv
Resolved 16 packages in 1.26s                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/24.87 KiB           
⠙ Preparing packages... (0/1)---------- 16.00 KiB/24.87 KiB         
⠙ Preparing packages... (0/1)---------- 24.87 KiB/24.87 KiB         
Prepared 1 package in 154ms                                                  
Installed 1 package in 3ms                                  
 + pyngrok==7.4.1


In [64]:
import nest_asyncio
from typing import Optional
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel
from pyngrok import ngrok

In [65]:
nest_asyncio.apply()

In [ ]:
class InboundEmail(BaseModel):
    subject: str
    from_email: str
    to_email: str
    text: Optional[str] = ""
    html: Optional[str] = ""
    thread_id: Optional[str] = None  # keep conversation context if you store it

#### 1.4 FastAPI app + endpoint

In [70]:
app = FastAPI()

@app.post("/sendgrid/inbound")
async def receive_inbound(payload: InboundEmail):
    message_body = payload.text or payload.html or ""
    router_prompt = (
        f"From: {payload.from_email}\nTo: {payload.to_email}\n"
        f"Subject: {payload.subject}\n\n{message_body}"
    )

    router_result = await Runner.run(reply_router, router_prompt)
    decision = router_result.final_output

    conversation_log.append(
        {
            "thread_id": payload.thread_id,
            "from": payload.from_email,
            "subject": payload.subject,
            "body": message_body,
            "router": decision,
        }
    )

    next_step = decision.get("next_step")
    if next_step == "auto_reply":
        reply_context = (
            f"Original message:\n{message_body}\n\n"
            f"Classification: {decision['classification']}\n"
            f"Summary: {decision['summary']}"
        )
        closer_result = await Runner.run(
            closer_agent,
            input=reply_context,
        )
        # Use the send_email tool through the agent response
        response_payload = closer_result.final_output
    else:
        response_payload = {
            "status": "queued_for_human",
            "notes": decision["summary"],
        }

    return JSONResponse(
        {
            "router": decision,
            "response": response_payload,
        }
    )

#### 1.5 Launch server + expose URL 

In [71]:
public_url = ngrok.connect(8000, "http", bind_tls=True).public_url
print(f"Inbound webhook URL for SendGrid Inbound Parse: {public_url}/sendgrid/inbound")

t=2025-11-03T12:58:22-0400 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  https://ngrok.com/docs/errors/err_ngrok_4018
ERROR:  


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

#### 1.6 Start uvicorn in the background 

In [ ]:
import uvicorn
server = uvicorn.Server(
    config=uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
)
await server.serve()

#### 1.7 Local test before wiring SendGrid

In [ ]:
client = TestClient(app)

mock_payload = {
    "subject": "Re: ComplAI follow-up",
    "from_email": "ceo@example.com",
    "to_email": "alice@complai.ai",
    "text": "Sounds good. Can you send over pricing and references?",
}

response = client.post("/sendgrid/inbound", json=mock_payload)
response.json()

### Testing/Demo for Agent-to-Agent response Simulation

In [ ]:
# Reuse send_email/send_html_email already defined earlier
reply_router = Agent(
    name="Reply Router",
    instructions="""
You triage inbound replies. Output JSON with keys:
- classification: one of ["positive", "neutral", "negative", "unsubscribe", "question"]
- summary: <=280 character recap
- next_step: either "auto_reply" or "human_review"
""",
    model="gpt-4o-mini"
)

closer_agent = Agent(
    name="Closer Agent",
    instructions="""
You write courteous follow-up replies as Alice from ComplAI.
Only respond if the router says next_step == auto_reply.

Steps:
1. Draft a plain text response body (no subject needed)
2. Hand off the email body to the Email Manager agent to format and send
3. Confirm the handoff was completed

Always hand off to Email Manager - do not try to send directly.
""",
    handoffs=[emailer_agent],  # Changed from tools to handoffs
    model="gpt-4o-mini"
)


conversation_log: list[dict] = []  # simple in-memory store; swap for DB in production

#### Agent-to-Agent Simulation (prospect_agent → reply_router → closer_agent)

##### 1.3 Helper models + agents


In [77]:
prospect_agent = Agent(
    name="Prospect Simulator",
    instructions="""
You role-play a busy CEO receiving a cold email from ComplAI.
Respond with one of these tones based on the user's request:
- positive: interested, asks for pricing/demo
- neutral: polite brush-off
- negative: annoyed, threatens spam report
- question: asks specific technical questions
Keep replies realistic, 2-4 sentences.
""",
    model="gpt-4o-mini"
)

In [78]:
# 1. Sales agent sends email
cold_email = await Runner.run(sales_agent1, "Write cold email to CEO")

# 2. Prospect "replies" (simulate)
prospect_reply = await Runner.run(
    prospect_agent, 
    f"You received:\n{cold_email.final_output}\n\nRespond with a 'positive' tone."
)

# 3. Router classifies
router_result = await Runner.run(reply_router, prospect_reply.final_output)

# 4. Closer drafts response if auto_reply
if "auto_reply" in router_result.final_output:
    closer_result = await Runner.run(closer_agent, router_result.final_output)
    print("Automated response sent:", closer_result.final_output)

Verified it works, so now put as One Unified Workflow as extension of Automated SDR

In [79]:
# UNIFIED WORKFLOW: Automated SDR with Prospect Simulation & Reply Handling

# First, create a conversation orchestrator that manages the full loop
conversation_orchestrator_instructions = """
You are a Conversation Orchestrator for ComplAI's automated SDR system.

Your workflow:
1. Use the sales_manager tool to generate and send the initial cold email
2. Use the prospect_simulator tool to generate a realistic reply
3. Use the reply_handler tool to classify and respond to the prospect's reply

Execute all three steps in sequence. Report a summary at the end.
"""

# Convert existing agents to tools
prospect_tool = prospect_agent.as_tool(
    tool_name="prospect_simulator",
    tool_description="Simulate a prospect's reply to a cold email with specified tone (positive/neutral/negative/question)"
)

# Create a reply handler agent that uses router + closer
reply_handler_instructions = """
You handle prospect replies in two steps:
1. First, classify the reply using reply_router to determine intent and next_step
2. If next_step is "auto_reply", draft and SEND the response using closer_agent
3. Return a summary of what happened

Always invoke the tools to do the work.
"""

reply_handler = Agent(
    name="Reply Handler",
    instructions=reply_handler_instructions,
    tools=[
        reply_router.as_tool("reply_router", "Classify prospect reply intent"),
        closer_agent.as_tool("closer_agent", "Draft and send automated follow-up email")
    ],
    model="gpt-4o-mini"
)

reply_handler_tool = reply_handler.as_tool(
    tool_name="reply_handler",
    tool_description="Triage a prospect reply and send automated response if appropriate"
)

# Create the orchestrator
conversation_orchestrator = Agent(
    name="Conversation Orchestrator",
    instructions=conversation_orchestrator_instructions,
    tools=[
        sales_manager.as_tool("sales_manager", "Generate and send initial cold email"),
        prospect_tool,
        reply_handler_tool
    ],
    model="gpt-4o-mini"
)

# Run the complete workflow in ONE trace
message = """
Execute the full SDR workflow:
1. Have sales_manager send a cold email to 'Dear CEO'
2. Have prospect_simulator generate a 'positive' tone reply to that email
3. Have reply_handler process the reply and send an automated response
"""

with trace("Complete SDR with Reply Automation"):
    result = await Runner.run(conversation_orchestrator, message)
    print("\n=== WORKFLOW COMPLETE ===")
    print(result.final_output)

CancelledError: 

Above didnt work because called sales_manager as a tool - When an agent is converted to a tool with .as_tool(), the framework treats it as a function call that should return quickly. Handoffs within that tool might not fully complete before control returns to the parent agent.

In [ ]:
# Step 1: Send initial cold email
print("=== STEP 1: Sending Initial Cold Email ===")
with trace("Complete SDR with Reply Automation - Step 1: Send Email"):
    cold_email_result = await Runner.run(
        sales_manager, 
        "Send out a cold sales email addressed to Dear CEO from Alice"
    )

# Extract the email content from sales_manager's execution
sent_email = cold_email_result.final_output
# (You may need to inspect cold_email_result to find where the email body is stored)
print(f"Email sent. Result: {cold_email_result.final_output[:200]}...")

# Step 2: Prospect simulation + Reply handling in one trace
print("\n=== STEP 2: Simulating Reply & Automated Response ===")

reply_flow_instructions = """
You orchestrate the prospect reply and response workflow.

Steps:
1. Use prospect_simulator to generate a realistic 'positive' tone reply
2. Use reply_handler to classify and respond to that reply

Execute both steps. The prospect_simulator will generate a simulated CEO reply,
and reply_handler will automatically send an appropriate response.
"""

reply_flow_agent = Agent(
    name="Reply Flow Agent",
    instructions=reply_flow_instructions,
    tools=[
        prospect_tool,
        reply_handler_tool
    ],
    model="gpt-4o-mini"
)

with trace("Complete SDR with Reply Automation - Step 2: Reply Flow"):
    reply_result = await Runner.run(
        reply_flow_agent,
        "Execute the reply simulation and automated response workflow"
    )
    print("\n=== WORKFLOW COMPLETE ===")
    print(reply_result.final_output)

=== STEP 1: Sending Initial Cold Email ===
Email sent. Result: I’ve prepared the email draft for sending. It will be processed by the Email Manager shortly. If you need any adjustments or additional information, feel free to ask!...

=== STEP 2: Simulating Reply & Automated Response ===

=== WORKFLOW COMPLETE ===
The prospect expressed interest and requested details on pricing and availability for a demo. Would you like assistance crafting a follow-up response to provide them with that information?


In [ ]:
# Agent that checks Gmail and handles real replies
real_reply_checker = Agent(
    name="Real Reply Checker",
    instructions="""
    You check Gmail for actual replies to sent emails.
    Use the check_gmail_for_replies tool to poll the inbox.
    If a reply is found, pass it to the reply_handler.
    If no reply is found, report that to the user.
    """,
    tools=[check_gmail_for_replies, reply_handler_tool],
    model="gpt-4o-mini"
)

In [ ]:
# Assuming you have defined the reply_router agent
# and the new check_gmail_for_replies_by_thread tool

real_reply_checker = Agent(
    name="Real Reply Checker",
    instructions="""
    You check Gmail for actual replies to sent emails.
    Use the 'check_gmail_for_replies_by_thread' tool with the provided thread_id 
    to poll the inbox.
    
    If a reply is found:
    1. Log the full reply data to the conversation_log.
    2. Pass the full reply data (including reply_found: True, body, thread_id, etc.) 
       to the **Reply Router** handoff for triage and classification.
       
    If no reply is found, report that to the user.
    """,
    tools=[check_gmail_for_replies_by_thread], # Update tool name
    handoffs=[reply_router], # Pass the Reply Router as a handoff
    model="gpt-4o-mini"
)

# How the workflow runs now (assuming cold_email_result contains the thread_id)

# 1. Capture Thread ID (from your send function, not fully shown, but crucial)
# sent_thread_id = cold_email_result['thread_id'] 

# 2. Wait for and handle real replies
print("\n=== STEP 2: Waiting for Real Reply (30 seconds) ===")

with trace("Real Email Reply Flow"):
    reply_result = await Runner.run(
        real_reply_checker,
        f"Check Gmail for a reply in thread {sent_thread_id}" # Agent gets the thread ID
    )
    # ...

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.